### Export ``.kml`` to csv 

Import bs library and parse given file

In [21]:
from bs4 import BeautifulSoup as b
with open('Аптеки.kml', 'r') as file:
    content = file.read()

soup = b(content, "lxml")

Load names and coordinates from the xml-like ``.kml`` file. Change the coordinates to google maps format.

In [22]:
names =  [ values.text for values in soup.findAll("name")][1:]

def format_coordinates(coordinates):
    splitted=coordinates.split(',')
    return [float(splitted[1]), float(splitted[0])]

latitude, longitude = zip(*[format_coordinates(values.text.strip()) for values in soup.findAll("coordinates")])

Make a pandas dataframe with columns 'name' (0), 'latitude' (1), 'longitude' (2). Drop the repeating coordinates. Save it to ``.csv``.

In [23]:
import itertools
import pandas as pd

data = [item for item in itertools.zip_longest(names, latitude, longitude)] 
df  = pd.DataFrame(data=data)
df = df.drop_duplicates(1)
df.to_csv("аптеки.csv",index=False, header=None)

In [ ]:
df.head()

In [25]:
df.shape

(79, 3)

In [26]:
df.describe()

,1,2
count,79.000000,79.000000
mean,55.727672,37.585877
std,0.063167,0.077853
min,55.611650,37.329102
25%,55.688964,37.548672
50%,55.712886,37.579664
75%,55.730929,37.609719
max,55.895301,37.930903


In [ ]:
!head аптеки.csv

### Export all the records in the one file 

In [84]:
import pandas as pd

df0 = pd.read_csv('аптеки.csv', header=None)
df1 = pd.read_csv('супермаркеты.csv', header=None)
df = df0.append(df1, ignore_index=True)

In [85]:
df.shape

(99, 3)

In [86]:
df.to_csv("base_drugstores_supermarkets.csv",index=False, header=None)

# Calculations of the nearest point 

In [60]:
from math import cos, asin, sqrt

def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(a))

def closest(data, v):
    return min(data, key=lambda p: distance(v['lat'],v['lon'],p['lat'],p['lon']))

tempDataList = [{'lat': 39.7612992, 'lon': -86.1519681}, 
                {'lat': 39.762241,  'lon': -86.158436 }, 
                {'lat': 39.7622292, 'lon': -86.1578917}]

v = {'lat': 39.7622290, 'lon': -86.1519750}
print(closest(tempDataList, v))

{'lat': 39.7612992, 'lon': -86.1519681}


In [106]:
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(a))

def closest(data, latitude, longitude):
    return min(data, key=lambda p: distance(v['lat'],v['lon'],p['lat'],p['lon']))

latitude, longitude = 55.7109579,37.6019022  # Ordzho
#latitude, longitude = 55.7131761,37.6042411

distances = df.apply(lambda row: distance(latitude, longitude, row[1], row[2]), axis=1)
top3 = distances.nsmallest(3).index.values
print(df.iloc[top3])

                 0          1          2
49       Медиофарм  55.709873  37.599971
87        Магнолия  55.709873  37.599971
47  Медбиолайн ООО  55.710121  37.599109
